In [3]:
using RCall
using Mosek
using JuMP
using CSV

LoadError: [91mArgumentError: Module Mosek not found in current path.
Run `Pkg.add("Mosek")` to install the Mosek package.[39m

In [5]:
L = Array{Float64,2}(CSV.read("../data/sample5000x20.txt", nullable = false, header = false, delim = ' '));

LoadError: [91mUndefVarError: CSV not defined[39m

In [ ]:
# vanilla interior point method for the primal
function vanilla_primal(L)
    n,m = size(L);
    mod = Model(solver=MosekSolver(QUIET = true));
    @variable(mod, x[1:m] >= 0, start = 1/m);
    @NLobjective(mod, Min, -sum(log(sum(L[i,j]*x[j] for j = 1:m)) for i = 1:n)/n);
    @constraint(mod, sum(x) == 1)
    solve(mod);
    x = getvalue(x);
    x[x.< 1e-3] = 0;
    return sparse(x/sum(x))
end

In [ ]:
# vanilla interior point method for the primal
function vanilla_boxed(L)
    n,m = size(L);
    mod = Model(solver=MosekSolver(QUIET = true));
    @variable(mod, x[1:m] >= 0, start = 1/m);
    @NLobjective(mod, Min, -sum(log(sum(L[i,j]*x[j] for j = 1:m)) for i = 1:n)/n + sum(x[i] for i = 1:m));
    solve(mod);
    x = getvalue(x);
    x[x.< 1e-3] = 0;
    return sparse(x/sum(x))
end

In [ ]:
# vanilla interior point method for the primal
function vanilla_dual(L)
    n,m = size(L);
    mod = Model(solver=MosekSolver(QUIET = true));
    @variable(mod, y[1:n] >= 0, start = 1/n);
    @NLobjective(mod, Min, -sum(log(y[i]) for i = 1:n)/n);
    @constraint(mod, ic, L'*y .<= 1)
    solve(mod);
    x = -getdual(ic);
    x[x.< 1e-3] = 0;
    return sparse(x/sum(x))
end

In [ ]:
function REBayes(L)
    @rput L;
    R"require(REBayes);
    t_rebayes = system.time(res <- KWDual(L, rep(1,dim(L)[2]), rep(1,dim(L)[1])/dim(L)[1]))[3];
    res$f[res$f < 1e-4] = 0
    x_rebayes = res$f / sum(res$f)"
    @rget x_rebayes;
    return sparse(x_rebayes)
end

In [ ]:
@time vanilla_dual(L)

In [ ]:
@time vanilla_primal(L)

In [ ]:
@time vanilla_boxed(L)

In [ ]:
@time REBayes(L)

In [ ]:
# L = Array{Float64,2}(CSV.read("../data/sample100000x100.txt", nullable = false, header = false, delim = ' '));
# warning : the following code might be extremely slow (with this large L)
# @time vanilla_primal(L)
# @time vanilla_dual(L)